# ДЗ №1 Ранжирование


Предисловие:

Я потратил много времени на поиск алгоритмов и способов их реализации в Python, в итоге остановился на библиотеке LightGBM из-за наиболее понятной документации и многочисленных примеров реализации задачи ранжирования в интернете

LightGBM - это разработанный Microsoft алгоритм градиентного бустинга, который широко используется в машинном обучении для решения различных задач, включая классификацию и регрессию. Он использует алгоритмы обучения на основе деревьев и у него есть функции специально созданные под ранжирование.

Конкретно в ранжировании работает LGBMRanker, который использует **pairwise ranking loss function** для оптимизации способности модели правильно ранжировать пары образцов. Функция потерь (loss function) основана на относительном упорядочивании меток релевантности в каждой паре.

В работе я опираюсь на [статью 20 года "Learning-to-rank with LightGBM"](https://tamaracucumides.medium.com/learning-to-rank-with-lightgbm-code-example-in-python-843bd7b44574) - в основном, все шаги я делаю опираясь на логику и код продемонстрированный Tamara Alexandra Cucumides


## Предварительная подготовка данных



### Загрузка библиотек, метрик, алгоритма

Загружаем и устанавливаем

In [ ]:
pip install lightgbm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# алгоритм
import lightgbm as lgb
from sklearn.model_selection import train_test_split

Метрики из классной работы:

In [ ]:
# Метрики - оценка качества модели

def dcg(relevances, k=10):
    #Discounted cumulative gain at k (DCG)
    relevances = np.asarray(relevances)[:k]
    n_relevances = len(relevances)
    gain = 2**relevances - 1
    # в формуле discounts + 2 т.к. нумерация начинается с 0
    discounts = np.log2(np.arange(n_relevances) + 2)
    return np.sum(gain / discounts)


def ndcg(relevances, k=10):
    #Normalized discounted cumulative gain (NDGC)
    best_dcg = dcg(sorted(relevances, reverse=True), k)
    if best_dcg == 0:
        return 0.

    return dcg(relevances, k) / best_dcg

def mean_ndcg(y_true, y_pred, query_ids, k=10):
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    query_ids = np.asarray(query_ids)
    ndcg_scores = []
    previous_qid = query_ids[0]
    previous_loc = 0
    for loc, qid in enumerate(query_ids):
        if previous_qid != qid:
            chunk = slice(previous_loc, loc)
            ranked_relevances = y_true[chunk][np.argsort(y_pred[chunk])[::-1]]
            ndcg_scores.append(ndcg(ranked_relevances, k=k))
            previous_loc = loc
        previous_qid = qid

    chunk = slice(previous_loc, loc + 1)
    ranked_relevances = y_true[chunk][np.argsort(y_pred[chunk])[::-1]]
    ndcg_scores.append(ndcg(ranked_relevances, k=k))
    return np.mean(ndcg_scores)

Бинарный датасет не нужен...

In [ ]:
train_Data = pd.read_csv('trainData.csv')
test_Data = pd.read_csv('testData.csv')

### Просмотр датасета, выделение тренировочной и тестовой выборок

In [ ]:
# освежим память, посмотрим на структуру данных

train_Data.head()

,relevance,qid,tf,idf,length,bm25,pagerank,inlink,outlink,slash,urlLength,childPage
0,0,11909,0.048537,0,0.054362,0.000000,0.000000,0.000000,0,0.208262,0.089286,1.000000
1,0,11909,0.000000,0,0.000000,0.000000,0.000000,0.080000,0,0.000000,0.000000,0.000000
2,0,11909,0.014989,0,0.005346,1.000000,1.000000,1.000000,0,1.000000,1.000000,0.166667
3,1,11909,0.048180,0,0.016753,0.000000,1.000000,0.253333,0,0.040667,0.017857,0.000000
4,2,11909,0.254818,0,0.135242,0.615723,0.333333,0.253333,0,0.004727,0.017857,0.527778


In [ ]:
train_Data.describe()

,relevance,qid,tf,idf,length,bm25,pagerank,inlink,outlink,slash,urlLength,childPage
count,4345.000000,4345.000000,4345.000000,4345.0,4345.000000,4345.000000,4345.000000,4345.000000,4345.0,4345.000000,4345.000000,4345.000000
mean,0.288377,14655.368470,0.155558,0.0,0.143830,0.162427,0.430521,0.328358,0.0,0.094315,0.107947,0.146688
std,0.582444,1696.743141,0.261629,0.0,0.267957,0.298707,0.277040,0.263226,0.0,0.241515,0.246806,0.260414
min,0.000000,11909.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
25%,0.000000,13322.000000,0.009479,0.0,0.006013,0.000000,0.222222,0.140000,0.0,0.000082,0.000348,0.000000
50%,0.000000,14395.000000,0.037453,0.0,0.022998,0.000000,0.400000,0.253846,0.0,0.002567,0.007299,0.019185
75%,0.000000,15894.000000,0.157895,0.0,0.110476,0.224008,0.600000,0.446429,0.0,0.031513,0.055556,0.162393
max,2.000000,18218.000000,1.000000,0.0,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,1.000000,1.000000


In [ ]:
test_Data.head(1)

,relevance,qid,tf,idf,length,bm25,pagerank,inlink,outlink,slash,urlLength,childPage
0,0,10032,0.031802,0,0.007235,0.0,0.5,0.46875,0,1.0,1.0,0.153846


In [ ]:
test_Data.describe()

,relevance,qid,tf,idf,length,bm25,pagerank,inlink,outlink,slash,urlLength,childPage
count,2144.000000,2144.000000,2144.000000,2144.0,2144.000000,2144.000000,2144.000000,2144.000000,2144.0,2144.000000,2144.000000,2144.000000
mean,0.354478,10965.632463,0.155629,0.0,0.152869,0.155109,0.446207,0.324808,0.0,0.101652,0.106991,0.147661
std,0.627045,569.560325,0.260827,0.0,0.275793,0.298319,0.294574,0.268445,0.0,0.242510,0.247397,0.267505
min,0.000000,10032.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
25%,0.000000,10534.000000,0.009345,0.0,0.005653,0.000000,0.200000,0.126582,0.0,0.000193,0.000399,0.000000
50%,0.000000,10947.000000,0.037539,0.0,0.024056,0.000000,0.400000,0.246093,0.0,0.004060,0.007299,0.018868
75%,1.000000,11511.000000,0.160947,0.0,0.127577,0.156078,0.666667,0.460890,0.0,0.049570,0.047619,0.153846
max,2.000000,11893.000000,1.000000,0.0,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,1.000000,1.000000


- qid -- идентификатор запроса
- tf, idf -- насколько слова из запроса встречаются в документе
- BM25 -- метрика соответствия документа запросу

Выделим отдельно столбцы с релевантностью и id запроса из тренировочного набора данных - это будет тренировочная выборка

(все как в классной работе)

In [ ]:
X_train = train_Data.iloc[:, 2:]
y_train = train_Data.loc[:,"relevance"].values

qid_train = train_Data.groupby("qid")["qid"].count().to_numpy()

Теперь - тестовая для проверки качества алгоритма




In [ ]:
X_test = test_Data.iloc[:, 2:]
y_test = test_Data.loc[:,"relevance"].values

qid_test = test_Data.groupby("qid")["qid"].count().to_numpy()

Когда фитим модель, нужно указать validation sets (т.е. использовать не только тренировочную дату)

Автор статьи разделила датасет не только на тренировочную и тестовую, но и на валидационную группу специально для этой цели
Т.е. на самой тестовой выборке делала только предикт

Я сделал также и получил mean_ndcg = 0.568, что не сильно отличается, чем когда я использую тестовую выборку в качестве валидационной (что можно будет заметить позже)

Поэтому в дальнейшем коде я убрал эти манипуляции с валидационной выборкой и оставил 2 стандартные выборки.

In [ ]:
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=282)

# query_val = [X_val.shape[0]]
# query_train = [X_train.shape[0]]

# X_train.shape[0] - для проверки сколько отнялось на валидационный сет

## Запуск алгоритма ранжирования и оценка

In [ ]:
gbm = lgb.LGBMRanker(learning_rate=0.01)

# покопался в документации, дефолт = 0.1. Тем ниже скорость, тем лучше
# "гиперпараметр, определяющий порядок того, как мы будем корректировать наши весы с учётом функции потерь в градиентном спуске. Чем ниже величина, тем медленнее мы движемся по наклонной"

In [ ]:
model = gbm.fit(X_train, y_train, group=qid_train,
        eval_set=[(X_test, y_test)], eval_group=[qid_test],
        eval_at=[50, 100])

model

# eval_at - parameters are the k used to evaluate metric (nDCG@k) over the validation (test) set

# model1 = gbm.fit(X_train, y_train, group=qid_train, eval_set=[(X_test, y_test)], eval_group=[qid_test], eval_at=[10, 50, 100], early_stopping_rounds=50)
# early_stopping_rounds - parameter for early stopping so your model doesn’t overfit - у меня руинило дальнейший код

[1]	valid_0's ndcg@50: 0.619162	valid_0's ndcg@100: 0.62941
[2]	valid_0's ndcg@50: 0.631162	valid_0's ndcg@100: 0.639066
[3]	valid_0's ndcg@50: 0.644514	valid_0's ndcg@100: 0.65095
[4]	valid_0's ndcg@50: 0.648379	valid_0's ndcg@100: 0.654863
[5]	valid_0's ndcg@50: 0.63949	valid_0's ndcg@100: 0.647323
[6]	valid_0's ndcg@50: 0.641361	valid_0's ndcg@100: 0.648721
[7]	valid_0's ndcg@50: 0.64755	valid_0's ndcg@100: 0.65547
[8]	valid_0's ndcg@50: 0.647046	valid_0's ndcg@100: 0.65517
[9]	valid_0's ndcg@50: 0.649385	valid_0's ndcg@100: 0.657328
[10]	valid_0's ndcg@50: 0.645846	valid_0's ndcg@100: 0.653457
[11]	valid_0's ndcg@50: 0.648505	valid_0's ndcg@100: 0.656568
[12]	valid_0's ndcg@50: 0.64798	valid_0's ndcg@100: 0.655986
[13]	valid_0's ndcg@50: 0.65075	valid_0's ndcg@100: 0.659116
[14]	valid_0's ndcg@50: 0.648658	valid_0's ndcg@100: 0.657233
[15]	valid_0's ndcg@50: 0.644066	valid_0's ndcg@100: 0.652653
[16]	valid_0's ndcg@50: 0.643954	valid_0's ndcg@100: 0.652315
[17]	valid_0's ndcg@50: 0

LGBMRanker(learning_rate=0.01)

Взвешенная качества модели NDCG (на основе релевантности топ100 и топ50 элементов) показывает индикатор в районе 0.65

Как мы помним из лекции, при идеальном варианте ранжирования NDCG близится к 1. Поэтому можно сказать, что у нас получилось ранжирование лучше среднего.

In [ ]:
test_pred = model.predict(X_test)

qid_train = train_Data.loc[:,"qid"].values
qid_test = test_Data.loc[:,"qid"].values

mean_ndcg(y_true = y_test, y_pred = test_pred, query_ids = qid_test, k=10)

0.5810280037994932

Используя оценку NDCG на тестовых данных (на всех получившихся запросах, не берем топ100 или топ5) получаем результат ранжирования в 0.58.

Ожидаемо, что эта оценка получилась меньше 0.65, но не критично, поэтому можно сказать, что модель не оверфитнулась (переобучилась) и хорошо показывает себя на практике

## Расположение документов согласно построенной модели

Наконец, можно прикрепить результаты спрогнозированного ранкинга к нашим тестовым данным...

In [ ]:
test_Data['predicted_relevance'] = test_pred
test_Data.sort_values("predicted_relevance", ascending=False)

,relevance,qid,tf,idf,length,bm25,pagerank,inlink,outlink,slash,urlLength,childPage,predicted_relevance
505,0,10443,0.380238,0,0.287715,0.000000,0.400000,0.269231,0,0.080140,0.500000,0.000000,0.646102
835,1,10766,0.332378,0,0.269254,0.000000,0.166667,0.202020,0,0.000000,0.000000,0.061224,0.583118
1788,2,11596,0.492958,0,0.072572,1.000000,0.000000,0.054545,0,1.000000,1.000000,0.947368,0.552249
157,0,10078,0.043750,0,0.012682,0.650693,0.666667,0.473118,0,0.000448,0.001751,0.000000,0.536082
544,2,10563,0.179718,0,0.165532,1.000000,0.200000,0.510204,0,0.137215,1.000000,0.125874,0.528480
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1594,1,11503,0.000000,0,0.102547,0.000000,0.600000,0.281250,0,1.000000,0.100000,0.076923,-1.172387
1231,0,11068,0.000000,0,0.037054,0.000000,1.000000,0.600000,0,0.021601,0.000000,0.216667,-1.172387
1741,0,11577,0.000000,0,0.123217,0.000000,0.250000,0.047619,0,0.002805,0.004975,0.750000,-1.178579
1164,0,11039,0.000000,0,0.088341,0.104647,0.500000,0.260417,0,0.044580,0.000000,0.787879,-1.244421


... и сравнить на рандомном запросе

In [ ]:
np.unique(qid_test)

array([10032, 10036, 10056, 10066, 10078, 10105, 10108, 10129, 10154,
       10164, 10173, 10178, 10197, 10206, 10215, 10220, 10258, 10264,
       10266, 10279, 10294, 10302, 10340, 10351, 10402, 10418, 10442,
       10443, 10480, 10494, 10534, 10563, 10575, 10581, 10584, 10599,
       10644, 10680, 10686, 10694, 10711, 10753, 10766, 10789, 10797,
       10800, 10820, 10875, 10889, 10915, 10934, 10941, 10943, 10947,
       10989, 10994, 11034, 11039, 11041, 11052, 11068, 11069, 11081,
       11092, 11125, 11165, 11181, 11208, 11215, 11223, 11282, 11292,
       11297, 11305, 11386, 11387, 11446, 11457, 11488, 11494, 11495,
       11503, 11511, 11531, 11553, 11565, 11577, 11581, 11596, 11624,
       11639, 11673, 11725, 11739, 11743, 11759, 11777, 11843, 11889,
       11893])

In [ ]:
# тестовые данные

y_test[qid_test == 10563]

array([0, 2, 0, 2, 0, 0, 0, 0])

На реальных данных ранк 2ой и 4ой переменной (выражаясь питоновским языком, 1 и 3) одинаков и предпочителен перед всеми остальными вариантами (равнозначными, 0)




In [ ]:
# спрогнозированный ранк

test_pred[qid_test == 10563]

array([-1.13823863,  0.37796881, -0.35136899,  0.52848008, -0.61849522,
        0.09402199, -0.48625823, -0.27731957])

Исходя из прогноза следует, что как раз эти самые 2ая и 4ая переменные имеют наибольший ранк! А остальные переменные даже имеют отрицательный (или очень близкий к нулю) вес, что соответствует реальной картине мира

Конкретно на этом запросе модель успешно справилась со своей задачей. Ура!